<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Reinforcement Learning for Finance

**Chapter 08 &mdash; Dynamic Asset Allocation (Three Asset Case)**

&copy; Dr. Yves J. Hilpisch

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>

### Please use the "Python 3.10, Tensorflow 2.10" kernel.

In [ ]:
!git clone https://github.com/tpq-classes/rl_4_finance.git
import sys
sys.path.append('rl_4_finance')


In [ ]:
import os
import warnings
warnings.simplefilter('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
%run assetallocation.py

In [ ]:
%config InlineBackend.figure_format = 'svg'

In [ ]:
days = 2 * 252

In [ ]:
random.seed(100)

In [ ]:
# 1 = X, 2 = Y, 3 = Z
investing = Investing('.SPX', '.VIX', 'EUR=', steps=days)

In [ ]:
investing.data.plot(lw=1, style=['g--', 'b:', 'm-.']);

In [ ]:
tf.random.set_seed(500)

In [ ]:
agent = InvestingAgent('3AC', feature=None, n_features=6,
                     env=investing, hu=128, lr=0.0001)

In [ ]:
episodes = 250

In [ ]:
%time agent.learn(episodes)

In [ ]:
agent.epsilon

In [ ]:
agent.env.portfolios = pd.DataFrame()

In [ ]:
%time agent.test(10)

In [ ]:
agent.env.portfolios.groupby('e')[
    ['xt', 'yt', 'zt']].mean().mean()

In [ ]:
agent.env.portfolios.groupby('e')[
    ['Xt', 'Yt', 'Zt', 'pv']].last().mean()

In [ ]:
def get_r(n):
    r = agent.env.portfolios[
        agent.env.portfolios['e'] == n
        ].set_index('date')
    return r

In [ ]:
n = min(agent.env.portfolios['e'])
# n += 1
n

In [ ]:
r = get_r(n)

In [ ]:
r[['xt', 'yt', 'zt']].mean()

In [ ]:
r[['xt', 'yt', 'zt']].std()

In [ ]:
r[['xt', 'yt', 'zt']].plot(
    title='POSITIONS[%]',
    style=['g--', 'b:', 'm-.'],
    lw=1, grid=True);

In [ ]:
cols = ['Xt', 'Yt', 'Zt', 'pv']

In [ ]:
sub = r[cols]

In [ ]:
rets = sub.iloc[-1] / sub.iloc[0] - 1
rets

In [ ]:
stds = sub.pct_change().std() * math.sqrt(252)
stds

In [ ]:
rets / stds

In [ ]:
sub.plot(style=['g--', 'b:', 'm-.', 'r-'], lw=1);

In [ ]:
sharpe = pd.DataFrame()

In [ ]:
def calculate_sr():
    for n in set(investing.portfolios['e']):
        r = get_r(n)
        sub = r[cols]
        rets = sub.iloc[-1] / sub.iloc[0] - 1
        stds = sub.pct_change().std() * math.sqrt(252)
        sharpe[n] = rets / stds

In [ ]:
calculate_sr()

In [ ]:
sharpe.round(2)

In [ ]:
sharpe.mean(axis=1)

In [ ]:
((sharpe.loc['pv'] > sharpe.loc['Xt']) &
 (sharpe.loc['pv'] > sharpe.loc['Yt']) &
 (sharpe.loc['pv'] > sharpe.loc['Zt'])).value_counts()

## Equally Weighted Portfolio

In [ ]:
agent.opt_action = lambda state: np.ones(3) / 3

In [ ]:
agent.env.portfolios = pd.DataFrame()

In [ ]:
%time agent.test(10)

In [ ]:
sharpe = pd.DataFrame()

In [ ]:
calculate_sr()

In [ ]:
sharpe.round(2)

In [ ]:
sharpe.mean(axis=1)

In [ ]:
((sharpe.loc['pv'] > sharpe.loc['Xt']) &
 (sharpe.loc['pv'] > sharpe.loc['Yt']) &
 (sharpe.loc['pv'] > sharpe.loc['Zt'])).value_counts()

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>